
# Glue Studio Notebook
You are now running a **Glue Studio** notebook; before you can start using your notebook you *must* start an interactive session.

## Available Magics
|          Magic              |   Type       |                                                                        Description                                                                        |
|-----------------------------|--------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------|
| %%configure                 |  Dictionary  |  A json-formatted dictionary consisting of all configuration parameters for a session. Each parameter can be specified here or through individual magics. |
| %profile                    |  String      |  Specify a profile in your aws configuration to use as the credentials provider.                                                                          |
| %iam_role                   |  String      |  Specify an IAM role to execute your session with.                                                                                                        |
| %region                     |  String      |  Specify the AWS region in which to initialize a session                                                                                                  |
| %session_id                 |  String      |  Returns the session ID for the running session.                                                                                                          |
| %connections                |  List        |  Specify a comma separated list of connections to use in the session.                                                                                     |
| %additional_python_modules  |  List        |  Comma separated list of pip packages, s3 paths or private pip arguments.                                                                                 |
| %extra_py_files             |  List        |  Comma separated list of additional Python files from S3.                                                                                                 |
| %extra_jars                 |  List        |  Comma separated list of additional Jars to include in the cluster.                                                                                       |
| %number_of_workers          |  Integer     |  The number of workers of a defined worker_type that are allocated when a job runs. worker_type must be set too.                                          |
| %worker_type                |  String      |  Standard, G.1X, *or* G.2X. number_of_workers must be set too. Default is G.1X                                                                            |
| %glue_version               |  String      |  The version of Glue to be used by this session. Currently, the only valid options are 2.0 and 3.0 (eg: %glue_version 2.0)                                |
| %security_config            |  String      |  Define a security configuration to be used with this session.                                                                                            |
| %sql                        |  String      |  Run SQL code. All lines after the initial %%sql magic will be passed as part of the SQL code.                                                            |
| %streaming                  |  String      |  Changes the session type to Glue Streaming.                                                                                                              |
| %etl                        |  String      |   Changes the session type to Glue ETL.                                                                                                                   |
| %status                     |              |  Returns the status of the current Glue session including its duration, configuration and executing user / role.                                          |
| %stop_session               |              |  Stops the current session.                                                                                                                               |
| %list_sessions              |              |  Lists all currently running sessions by name and ID.                                                                                                     |
| %spark_conf                 |  String      |  Specify custom spark configurations for your session. E.g. %spark_conf spark.serializer=org.apache.spark.serializer.KryoSerializer                       |

In [20]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)


data_bucket_url = "s3://spark.demo.data/"
lab_output_bucket_url = "s3://spark.labs.saves/"

# Exercise 1 Using the PySpark API

Add some code in the notebook as follows:

* Open the Macbeth.txt file from the lab bucket which is called spark-lab-data. This will return an RDD of strings, ie all the lines in the file. Assign the result to a variable named lines.

To do this you will need to use the glueContext variable since this has the API capability to read and write directly to and from S3. The API call will be something like:

```
lines = glueContext.read.text("s3://spark-lab-data/Macbeth.txt")
```

* Call lines.count() to count the number of items in the RDD (i.e. the number of lines in the file). Print the result.
* Call lines.first() to obtain the first item in the RDD (i.e. the first line in the file). Print the result.
* Call lines.filter() to filter the lines, and print the result. Note that the filter() function takes the column (which in this case is called value) and then a contains() function can be used to filter out the content. So the code would look something like:

```
witchLines = lines.filter(lines["value"].contains("Witch"))
```
Run your code block and you should see output like the following
* Number of lines: 4102
* First line: ACT I
* Witch lines: (A collection of RDD objects)

 
## Saving RDDs 
Modify the code so that it saves the witchLines RDD which you then can write to S3. 

The easiest way to complete this is to use the write function contained within the dataframe itself. Save the file to your S3 bucket with a path something like YOURBUCKET/lab1. 

The code would be something like this but the bucket name would be different.

```
witchLines.write.format('csv').option('header','false').save('s3://glue-nickt/lab1')
```

Now go to the S3 service in AWS and locate your bucket. There should now be a new file in there with your outpput in it. If you want to, you can download it and review the contents.


In [21]:


# read in the Macbeth file
lines = glueContext.read.text(data_bucket_url + "Macbeth.txt")
lines.show()

# see the datatype
print(type(lines))

# how many lines
numLines = lines.count()
print("Number of lines: %d" % numLines)

# print the first line
firstLine = lines.first()
print("First line: %s" % firstLine)

# find the lines about witches!
witchLines = lines.filter(lines["value"].contains("Witch"))
witchLines.show()

# write the witch lines back to s3
witchLines.write.format('csv').option('header','false').mode("overwrite").save(lab_output_bucket_url + "nick/lab1")


+--------------------+
|               value|
+--------------------+
|               ACT I|
|SCENE I. A desert...|
|                    |
|Thunder and light...|
|        First Witch |
|When shall we thr...|
|In thunder, light...|
|                    |
|       Second Witch |
|When the hurlybur...|
|When the battle's...|
|                    |
|        Third Witch |
|That will be ere ...|
|                    |
|        First Witch |
|    Where the place?|
|                    |
|       Second Witch |
|     Upon the heath.|
+--------------------+
only showing top 20 rows

<class 'pyspark.sql.dataframe.DataFrame'>
Number of lines: 4102
First line: ACT I
+--------------------+
|               value|
+--------------------+
|Thunder and light...|
|        First Witch |
|       Second Witch |
|        Third Witch |
|        First Witch |
|       Second Witch |
|        Third Witch |
|        First Witch |
|       Second Witch |
|        Third Witch |
|Thunder. Enter th...|
|        First Wit

# Exercise 2 RDD Operations Part 1

## Overview
In this lab you'll use various RDD transformation methods to manipulate data from several simple text files.

## Roadmap
There are 3 exercises in this lab, of which the last exercise is "if time permits". Here is a brief summary of the tasks you will perform in each exercise; more detailed instructions follow later:
1. Mapping and filtering data
2. Performing set-based operations
3. (If Time Permits) Additional transformation suggestions

## Part 1: Mapping and filtering data
In this exercise you will perform various mapping and filtering operations on RDD objects. You will do all your work in app1.py, so open this file in the text editor now. The application makes use of data from two text files:
* klm.txt – contains a partial list of airports that KLM flies into.
* norwegian.txt – contains a partial list of airports that Norwegian Airlines flies into.

Follow the comments in the codeblock below to complete this exercise. After each step, save the Notebook and use the play button to test. Make sure you have run the first code block in the notebook first to ensure that the glueContext and sparkContext is all configured.

In [22]:
klmAirports = glueContext.read.text(data_bucket_url + "klm.txt").rdd
norAirports = glueContext.read.text(data_bucket_url + "norwegian.txt").rdd


# In each of the following statements, replace "None" with a suitable call to a PySpark API function...
# We've done the first one for you, to get you started...

# Get KLM airports in uppercase.
klmUpperCaseAirports = klmAirports.map(lambda airport: airport["value"].upper()).collect()
print("\nKLM airports in uppercase: %s" % klmUpperCaseAirports)

# Get KLM airports that start with "L" (hint, Python strings have a startswith() method).
klmLAirports = klmAirports.filter(lambda airport: airport["value"].startswith("L")).collect()
print("\nKLM airports that start with 'L': %s" % klmLAirports)

# Get the union of all airports.
allAirports = klmAirports.union(norAirports).collect()
print("\nUnion of all airports: %s" % allAirports)

# Get all distinct airports.
distinctAirports = klmAirports.union(norAirports).distinct().collect()
print("\nAll distinct airports: %s" % distinctAirports)

# Get airports in common.
commonAirports = klmAirports.intersection(norAirports).collect()
print("\nAirports in common: %s" % commonAirports)

# Get airports served by KLM but not Norwegian.
klmOnlyAirports = klmAirports.subtract(norAirports).collect()
print("\nAirports served by KLM but not Norwegian: %s" % klmOnlyAirports)



KLM airports in uppercase: ['LONDON HEATHROW', 'LONDON GATWICK', 'AMSTERDAM', 'CARDIFF', 'OSLO', 'DUBAI', 'SINGAPORE', 'BERGEN', 'BRISTOL', 'PARIS', 'LONDON CITY', 'LUTON', 'LIVERPOOL']

KLM airports that start with 'L': [Row(value='London Heathrow'), Row(value='London Gatwick'), Row(value='London City'), Row(value='Luton'), Row(value='Liverpool')]

Union of all airports: [Row(value='London Heathrow'), Row(value='London Gatwick'), Row(value='Amsterdam'), Row(value='Cardiff'), Row(value='Oslo'), Row(value='Dubai'), Row(value='Singapore'), Row(value='Bergen'), Row(value='Bristol'), Row(value='Paris'), Row(value='London City'), Row(value='Luton'), Row(value='Liverpool'), Row(value='Oslo'), Row(value='Tromso'), Row(value='London Gatwick'), Row(value='Bergen'), Row(value='Molde'), Row(value='Trondheim')]

All distinct airports: [Row(value='Luton'), Row(value='Molde'), Row(value='Oslo'), Row(value='London City'), Row(value='Liverpool'), Row(value='Amsterdam'), Row(value='London Gatwick'), R

## Part 2: Performing set-based operations
In this exercise you will perform set-based operations on RDD objects. You will do all your work in the code block below. The application makes use of data from two text files:

* teams.txt – contains the names of Premier League teams, 2017-18 (happy days!)
* stadiums.txt – contains the stadiums of Premier League teams, 2017-18

Follow the comments in the code block below to complete this exercise. After each step, save the Notebook and click the play button (remembering to ensure that the block at the top of the notebook has executed. 

Here are some additional hints:
* To zip teams with stadiums, use the zip() method. You should obtain a collection of tuples such as (Arsenal, The Emirates), (Bournemouth, Vitality Stadium), etc.
* To get the Cartesian product of all teams, call the cartesian() method on the teams RDD. Also pass the teams RDD as a parameter, so that you perform a Cartesian product between all the teams. You should obtain a collection of tuples such as (Arsenal, Arsenal), (Arsenal, Bournemouth), (Arsenal, Burnley), etc.
* To get all the fixtures, it's similar to getting a Cartesian product but you need to filter-out the tuples where element [0] equals element [1]. E.g., tuples such as (Arsenal, Arsenal) should be excluded from the result. 


In [23]:
teams = glueContext.read.text(data_bucket_url + "teams.txt").rdd
stadiums = glueContext.read.text(data_bucket_url + "stadiums.txt").rdd

# In each of the following statements, replace "None" with a suitable call to a PySpark API function...

# Zip teams with stadiums.
teamsAndStadiums = teams.zip(stadiums).collect()
print("Teams and stadiums: %s" % teamsAndStadiums)

# Cartesian product of all teams.
cartesian = teams.cartesian(teams).collect()
print("Cartesian product of all teams: %s" % cartesian)

# Fixtures.
fixtures = teams.cartesian(teams).filter(lambda twoTeams: twoTeams[0] != twoTeams[1]).collect()
print("Fixtures: %s" % fixtures)



Teams and stadiums: [(Row(value='Arsenal'), Row(value='The Emirates')), (Row(value='Bournemouth'), Row(value='Vitality Stadium')), (Row(value='Brighton and Hove Albion'), Row(value='Falmer Stadium')), (Row(value='Burnley'), Row(value='Turf Moor')), (Row(value='Chelsea'), Row(value='Stamford Bridge')), (Row(value='Crystal Palace'), Row(value='Selhurst Park')), (Row(value='Everton'), Row(value='Goodison Park')), (Row(value='Huddersfield Town'), Row(value='King Power Stadium')), (Row(value='Leicester City'), Row(value='Kirklees Stadium')), (Row(value='Liverpool'), Row(value='Anfield')), (Row(value='Manchester City'), Row(value='The Etihad')), (Row(value='Manchester United'), Row(value='Old Trafford')), (Row(value='Newcastle United'), Row(value="St. James' Park")), (Row(value='Southampton'), Row(value="St Mary's Stadium")), (Row(value='Stoke City'), Row(value='Britannia Stadium')), (Row(value='Super Swans'), Row(value='Liberty Stadium')), (Row(value='Tottenham Hotspur'), Row(value='White H

## Part 3 (If time permits):  Additional transformation suggestions 
In this exercise you'll perform various transformation operations on data for some chemical elements in the periodic table. The data is located in elements.txt. For each element, we've specified the following information (we've used tabs as field separators):
* Atomic number
* Atomic mass
* Group number in the periodic table
* Period number in the periodic table
* Symbol 
* Name of element

You can review the periodic table here if you are a bit rusty on your chemistry!
https://sciencenotes.org/wp-content/uploads/2013/06/PeriodicTable-NoBackground2.png

We've defined a Python class named Element in the codeblock below to represent an element in the periodic table.  All the code you have to write will be in the block below that. Complete the application as per the comments in codeblock below the Element block. 


In [24]:
# Run  this block once to load the class
class Element:
    def __init__(self, str):
        arr = str.split("\t") 
        self.atomicNumber = int(arr[0])
        self.atomicMass = float(arr[1])
        self.group = int(arr[2])
        self.period = int(arr[3])
        self.symbol = arr[4]
        self.name = arr[5]

    def __repr__(self):
        return "%d | %f | %d | %d | %s | %s" % (self.atomicNumber, self.atomicMass, self.group, self.period, self.symbol, self.name)


In [ ]:
lines = glueContext.read.text(data_bucket_url + "elements.txt").rdd

print(lines.first())
# In each of the following statements, replace "None" with a suitable call to a PySpark API function...

# Map each line into an Element object. 
elements = lines.map(lambda line: Element(str(line["value"]))) 
print(elements.first())
    
# Group elements by period.
groupedByPeriod = elements.groupBy(lambda e: e.period).collect()
print("Elements grouped by period: %s" % groupedByPeriod)

# Create a dict, where the key is the element symbol, and the value is the element itself.
keyedBySymbol = elements.keyBy(lambda e: e.symbol).collect()
print("Elements keyed by symbol: %s" % keyedBySymbol)

# Sort elements by name.
sortedByName = elements.sortBy(lambda e: e.name).collect()
print("Elements sorted by name: %s" % sortedByName)

# Repartition elements into 5 partitions, and then add code to save in a directory named "partitionedElements".
repartitionedElements = elements.repartition(5)
print("Number of partitions: %d" % repartitionedElements.getNumPartitions())
# To save: repartitionedElements.saveAsTextFile(lab_output_bucket_url + "partitionedElements")

Row(value='1\t1.0079\t1\t1\tH\tHydrogen\t\t')
1 | 1.007900 | 1 | 1 | H | Hydrogen
Elements keyed by symbol: None
Elements sorted by name: None


# Exercise 3 RDD Operations Part 2

## Part 1: Mapping lines of text to a flat map of words
In the empty code box below. create an RDD named lines, containing all the lines of text from MacbethSnippet.txt which is a file in the labfile bucket used in the other code blocks.

Where indicated by the "Ex 1" comment, add code to create an RDD containing all the words in all the lines. Here are some hints:
* Use the RDD flatMap() function, whose purpose is to map an item into a sequence of sub-items. In this case, it will map a line into its constituent words…
* flatMap() takes a lambda expression as a parameter, indicating how to map each item into sub-items. Implement the lambda so that it splits a line at the space character (call the split(' ') function to do this).
* The flatMap() function returns all words, including words that are effectively empty. Call filter() to just keep the non-empty words. You'll need to write a lambda that tests if a word isn't empty… how can you test if a Python string isn't empty?

Finally, print the resultant RDD to the console, via the print() function. In order to see the actual contents of the RDD, call collect() on the RDD for simplicity.

Run the code block to verify that the application prints all the words found in the text file. 

In [26]:
lines = glueContext.read.text(data_bucket_url + "MacbethSnippet.txt").rdd

# Add your code for part 1 here.
words = lines.flatMap(lambda line:line["value"].split(' ')) \
             .filter(lambda word: word)
print("All words: %s" % words.collect())




All words: ['ACT', 'I', 'SCENE', 'I.', 'A', 'desert', 'place.', 'Thunder', 'and', 'lightning.', 'Enter', 'three', 'Witches', 'First', 'Witch', 'When', 'shall', 'we', 'three', 'meet', 'again', 'In', 'thunder,', 'lightning,', 'or', 'in', 'rain?', 'Second', 'Witch', 'When', 'the', "hurlyburly's", 'done,', 'When', 'the', "battle's", 'lost', 'and', 'won.', 'Third', 'Witch', 'That', 'will', 'be', 'ere', 'the', 'set', 'of', 'sun.', 'First', 'Witch', 'Where', 'the', 'place?', 'Second', 'Witch', 'Upon', 'the', 'heath.', 'Third', 'Witch', 'There', 'to', 'meet', 'with', 'Macbeth.', 'First', 'Witch', 'I', 'come,', 'Graymalkin!', 'Second', 'Witch', 'Paddock', 'calls.', 'Third', 'Witch', 'Anon.', 'ALL', 'Fair', 'is', 'foul,', 'and', 'foul', 'is', 'fair:', 'Hover', 'through', 'the', 'fog', 'and', 'filthy', 'air.', 'Exeunt', 'SCENE', 'II.', 'A', 'camp', 'near', 'Forres.', 'Alarum', 'within.', 'Enter', 'DUNCAN,', 'MALCOLM,', 'DONALBAIN,', 'LENNOX,', 'with', 'Attendants,', 'meeting', 'a', 'bleeding', 'S

## Part  2: Creating an RDD of (word, count) tuples
In the following code block, add code to create an RDD containing tuples of words and word counts (i.e. the first item in the tuple is a word, and the second item is the count of how many times that word appears). 

Follow these steps:
* Call map() to map each word into a (word, count) tuple where each count is 1 initially. 
* Call reduceByKey(), to group (key, value) tuples by key and then reduce the values into a single value. E.g. imagine you start off with the following (word, count) tuples:
    * ("do", 1)
    * ("you", 1)
    * ("know", 1)
    * ("the", 1)
    * ("muffin", 1)
    * ("man", 1)
    * ("the", 1)
    * ("muffin", 1)
    * ("man", 1)
    *  ("the", 1)
    * ("muffin", 1)
    * ("man", 1)

reduceByKey() first groups these tuples by key (i.e. word) as follows:
    * ("do", 1)
    * ("you", 1)
    * ("know", 1)
    * ("the", 1)	("the", 1)	("the", 1)
    * ("muffin", 1)	("muffin", 1) 	("muffin", 1)
    * ("man", 1)	("man", 1)	("man", 1)

For each different key, reduceByKey() calls the lambda expression that you provide, successively with each value. E.g. for the "muffin" key, it'll call your lambda 3 times, passing the values 1, 1, 1 each time. 

So, implement a lambda that accumulates the total word count for each word. The lambda receives two parameters: 
* The accumulated value so far (defaults to 0 on the first call)
* The value of the next item in the key group (e.g. the value 1)
* Finally call sortBy() to sort the (word, count) tuples by descending word count.
* Print the resultant RDD of (word, count) tuples.

Test your new code. 

In [27]:
# Add your code for Part 2 here
wordCounts = words.map(lambda word: (word, 1)) \
                  .reduceByKey( lambda accumulatedValue, thisValue: accumulatedValue + thisValue) \
                  .sortBy(lambda wc: wc[1], False)
print("All words and counts: %s" % wordCounts.collect())


All words and counts: [('the', 20), ('Witch', 9), ('to', 9), ('and', 9), ('I', 6), ('As', 6), ('his', 6), ('of', 6), ('with', 6), ('is', 5), ('a', 5), ('that', 4), ('Sergeant', 4), ('DUNCAN', 3), ('Second', 3), ('Third', 3), ('The', 3), ('When', 3), ('And', 3), ('he', 3), ('they', 3), ('First', 3), ('upon', 3), ('their', 2), ('whence', 2), ('brave', 2), ('Enter', 2), ('fortune,', 2), ('SCENE', 2), ('kerns', 2), ('three', 2), ('Which', 2), ('meet', 2), ('be', 2), ('Till', 2), ('A', 2), ('for', 2), ('from', 2), ('or', 2), ('my', 2), ('like', 2), ('king', 2), ('bloody', 2), ('in', 2), ('But', 2), ('ACT', 1), ('shall', 1), ('lightning,', 1), ('sergeant', 1), ('soldier', 1), ('Say', 1), ('rebel,', 1), ("brandish'd", 1), ('steel,', 1), ('him', 1), ('worthy', 1), ('skipping', 1), ('vantage,', 1), ('arms', 1), ('assault.', 1), ('cannons', 1), ('Except', 1), ('cry', 1), ('I.', 1), ('Graymalkin!', 1), ('Anon.', 1), ('Hover', 1), ('fog', 1), ('that?', 1), ('good', 1), ("unseam'd", 1), ('comfort',

## Part 3: Caching the RDD of (word, count) tuples

The exercises that follow will perform various actions on the RDD of (word, count) tuples. Under normal circumstances, Spark would re-evaluate all the intermediate steps to recreate the RDD ready for each action. This is inefficient, so add code to cache the RDD of (word, count) tuples before going any further.

In [28]:
# add your code for part 3 here
wordCounts.cache()


PythonRDD[336] at collect at <stdin>:4


## Part 4: Displaying the first 100 words

In the code box below, add code to display the first100 words. Here are some hints:
* First call take() to create an RDD containing just the first 100 items.
* Then pass the resultant RDD to the Python print() function.

---
**NOTE**
Aside: An alternative to take() is collect(). However, collect() might cause the Spark driver to run out of memory, because it fetches the entire RDD to a single machine. This is why take() is generally a safer option, because it limits the number of items being collated.

---

In [29]:
# Add code for part 4 here
print("First 100 words and counts: %s" % wordCounts.take(100))



First 100 words and counts: [('the', 20), ('Witch', 9), ('and', 9), ('to', 9), ('I', 6), ('As', 6), ('of', 6), ('with', 6), ('his', 6), ('is', 5), ('a', 5), ('that', 4), ('Sergeant', 4), ('And', 3), ('he', 3), ('they', 3), ('DUNCAN', 3), ('Second', 3), ('First', 3), ('Third', 3), ('upon', 3), ('The', 3), ('When', 3), ('their', 2), ('whence', 2), ('brave', 2), ('Enter', 2), ('SCENE', 2), ('kerns', 2), ('three', 2), ('Which', 2), ('meet', 2), ('be', 2), ('Till', 2), ('fortune,', 2), ('in', 2), ('But', 2), ('A', 2), ('for', 2), ('from', 2), ('or', 2), ('my', 2), ('like', 2), ('king', 2), ('bloody', 2), ('lightning.', 1), ('rain?', 1), ('Where', 1), ('bleeding', 1), ('revolt', 1), ('MALCOLM', 1), ('didst', 1), ('nature', 1), ("all's", 1), ('weak:', 1), ('Like', 1), ("valour's", 1), ('faced', 1), ('shook', 1), ('chaps,', 1), ('storms', 1), ('Mark,', 1), ("furbish'd", 1), ('were', 1), ('Golgotha,', 1), ('place.', 1), ('Fair', 1), ('air.', 1), ('Alarum', 1), ('report,', 1), ("'Gainst", 1), ('

## Part 5: Performing aggregation actions

In the code block below, add code to perform the following aggregation actions on the RDD of (word, count) tuples:
* Find the count of all items in the RDD, via the RDD count() method. This tells you the number of different words in MacbethSnippet.txt. 
* Find the most frequent word. Use the RDD max() method to do this. Note the following points about the max() method:
    * The max() method takes an optional lambda expression, which allows you to specify how to compare items… 
    * In our scenario, the RDD contains (word, count) tuples, and you want max() to compare element [1] in the tuples (i.e. the counts)…
    * Therefore, when you call max(), pass in a lambda that takes a (word, count) tuple and returns element [1] from the tuple. 
* In the same way, find the least frequent word. Use the RDD min() method to do this.

In [30]:
# add code for part 5 here

count = wordCounts.count()
max   = wordCounts.max(lambda wc: wc[1])
min   = wordCounts.min(lambda wc: wc[1])
print("Number of different words: %d" % count)
print("Most frequent word: %s, count: %d" % max)
print("Least frequent word: %s, count: %d" % min)

Number of different words: 294
Most frequent word: the, count: 20
Least frequent word: lightning., count: 1


## Part 6 (If time permits): Performing key-based actions
In the code block below, add code to lookup a word and find its count. 

You could for example set up a list of words that you would like to count.

Write code to iterate through the list. For each word, look it up in the (word, count) tuple. This will tell you the occurrence count for that word. Display the results.

In [31]:
# Add the code for part 6 here

wordsToCount = ["the", "a", "in"]

for w in wordsToCount:
    c = wordCounts.lookup(w)
    print("Word: %s, count: %s" % (w, c))

Word: the, count: [20]
Word: a, count: [5]
Word: in, count: [2]


## Part 7 (If time permits): Performing numeric actions

In the code block below, add code to perform the following numeric actions on the word counts:
* The sum of all counts (i.e. the total number of words)
* The average (mean) of all counts
* The standard deviation of all counts
* The variance of all counts

Note 1: You'll first need to create an RDD containing just the word counts but not the words themselves – how will you do this?
Note 2: Ensure that the process by which you map (word, count) tuples isn't repeated each time you invoke one of the action methods.


In [32]:
# Add code for part 7 here
counts = wordCounts.map(lambda wc: wc[1])
counts.cache()
sum = counts.sum()
mean = counts.mean()
stdev = counts.stdev()
variance = counts.variance()
print("Sum: %d, mean: %f, standard deviation: %f, variance: %f" % (sum, mean, stdev, variance))

Sum: 418, mean: 1.421769, standard deviation: 1.584041, variance: 2.509186


# Exercise 6 Getting Started with Spark SQL

## Overview
In this lab you’ll use Spark SQL to read data from a CSV file. This is very similar to reading data from a JSON file, which we covered during the chapter.

Roadmap
There are 5 exercises in this lab, of which the last exercise is "if time permits". Here is a brief summary of the tasks you will perform in each exercise; more detailed instructions follow later:
1. Reading data from a CSV file into a DataFrame
2. Specifying options when reading a file
3. Defining a schema for the data
4. Executing a SQL query
5. (If Time Permits) Generating computed columns
 
## Familiarization
From the lab bucket, download and review weather.csv in a text editor. This file contains real temperature and precipitation measurements for every day in Bergen in 2019 (spoiler alert, it was wet). All weather measurements are from Yr, delivered by the Norwegian Meteorological Institute and NRK. 

The CSV file has 365 rows, containing the data for every day from 1 January to 31 December. Each row has 4 values:
* The first value indicates the day number in a month (notice how this value wraps from 31 back to 1 as we move from January to February, for example).
* The next value contains the minimum temperature for a day, in degrees Celsius.
* The next value contains the maximum temperature for a day, in degrees Celsius.
* The final value contains the precipitation for a day, in mm.
 
## Part 1: Reading data from a CSV file into a DataFrame
Open app.py in a text editor and add code to read data from weather.csv into a Spark SQL DataFrame object in memory. It’s similar to how you read JSON (see the PowerPoint chapter for a reminder), except you call the csv() method instead:
```
df = glueContext.read.csv("s3://spark-lab-data/weather.csv")
```

Spark SQL parses each line of text in the file, using a comma as the default delimiter. It creates a DataFrame with 365 rows, with 4 columns per row. Display the DataFrame object as follows:
```
df.show()
```

Run the code as usual:
  
Note that the DataFrame’s columns are named _c0, _c1, _c2, and _c3 by default:

In [33]:
# Add code for part 1

# this loads the data in as a pyspark sql dataframe
df = glueContext.read.csv(data_bucket_url + "weather.csv")
df.show()




+---+----+---+----+
|_c0| _c1|_c2| _c3|
+---+----+---+----+
|  1| 3.1|9.7|25.2|
|  2| 0.6|4.8| 1.5|
|  3|-0.9|3.8| 0.3|
|  4| 2.9|8.2| 9.3|
|  5| 7.3|  8|31.5|
|  6|   4|7.5| 5.3|
|  7| 2.4|5.5| 6.1|
|  8| 3.4|  6|11.7|
|  9| 0.5|3.7|   0|
| 10| 1.1|5.2| 0.4|
| 11| 3.4|7.6| 7.4|
| 12| 1.9|3.9| 0.1|
| 13| 0.7|6.7|17.8|
| 14|-2.2|2.2| 6.6|
| 15|-0.7|  6|   5|
| 16| 1.6|6.4|12.9|
| 17|  -1|2.2| 8.2|
| 18|-3.6|4.2| 3.9|
| 19| 0.9|4.3| 4.5|
| 20|-1.2|3.3| 2.1|
+---+----+---+----+
only showing top 20 rows


   
## Part 2: Specifying options when reading a file
In the previous exercise your Python application loaded weather.csv into a DataFrame. The CSV file didn’t give any indication about the meaning of each field, so Spark SQL created columns named _c0, _c1, _c2, and _c3 by default.

An alternative approach is for the CSV file to include a “header line” at the top. We’ve done this in weatherWithHeader.csv – download this file from the S3 bucket and take a look in a text editor. Also note that the file happens to use semi-colons rather than commas as the field delimiter, just to make life interesting.

In order to read this file into a DataFrame, you have to specify a couple of options to PySpark:
* You need to tell PySpark to expect the header line.
* You need to tell PySpark that the field delimiter is a semi-colon rather than a comma.

This is how you specify options when reading a CSV file: 
```
df = glueContext.read \
                  .option(option-name-1, option-value-2) \
                  .option(option-name-2, option-value-2) \
                  .csv("s3://spark-lab-data/weatherWithHeader.csv")
```

Take a look at the online docs here to see what option names and values you need to specify:
https://spark.apache.org/docs/latest/sql-data-sources-csv.html

(Scroll down the website a couple of pages to see a list of all the option properties you can set. You’re looking for how to specify that a header line is expected, and that the delimiter is a semi-colon).
When you’ve modified your code, submit your application to PySpark again and verify it works properly. You should now see columnar output with proper column names.

In [34]:
# Add code for part 2
df = glueContext.read \
               .option("delimiter", ";") \
               .option("header", "true") \
               .csv(data_bucket_url + "weatherWithHeader.csv")
               
df.show()


+----------+-------+-------+-------------+
|DayOfMonth|MinTemp|MaxTemp|Precipitation|
+----------+-------+-------+-------------+
|         1|    3.1|    9.7|         25.2|
|         2|    0.6|    4.8|          1.5|
|         3|   -0.9|    3.8|          0.3|
|         4|    2.9|    8.2|          9.3|
|         5|    7.3|      8|         31.5|
|         6|      4|    7.5|          5.3|
|         7|    2.4|    5.5|          6.1|
|         8|    3.4|      6|         11.7|
|         9|    0.5|    3.7|            0|
|        10|    1.1|    5.2|          0.4|
|        11|    3.4|    7.6|          7.4|
|        12|    1.9|    3.9|          0.1|
|        13|    0.7|    6.7|         17.8|
|        14|   -2.2|    2.2|          6.6|
|        15|   -0.7|      6|            5|
|        16|    1.6|    6.4|         12.9|
|        17|     -1|    2.2|          8.2|
|        18|   -3.6|    4.2|          3.9|
|        19|    0.9|    4.3|          4.5|
|        20|   -1.2|    3.3|          2.1|
+----------

## Part 3: Defining a schema for the data
In your application as it stands, you don’t tell Spark SQL about the schema of the CSV data in advance. Therefore, Spark SQL has to scan the entire dataset initially so that it can infer the schema from the shape of the data.

If you know the schema, you can tell the Spark SQL in advance by calling the schema() method on the DataFrameReader. Modify your code to do this, then run the application again to make sure it all still works fine. You won’t see any changes in your application’s behaviour, but under the covers you’ve improved its efficiency 👍.

In [35]:
# Add code for part 3

from pyspark.sql.types import StructType, StructField, IntegerType, DoubleType

weatherSchema = StructType([
	StructField("DayOfMonth", IntegerType(), False),
	StructField("MinTemp", DoubleType(), False),
	StructField("MaxTemp", DoubleType(), False),
	StructField("Precipitation", DoubleType(), False)
])

df = glueContext.read \
               .schema(weatherSchema) \
               .option("delimiter", ";") \
               .option("header", "true") \
               .csv(data_bucket_url + "weatherWithHeader.csv")
               
df.show()

+----------+-------+-------+-------------+
|DayOfMonth|MinTemp|MaxTemp|Precipitation|
+----------+-------+-------+-------------+
|         1|    3.1|    9.7|         25.2|
|         2|    0.6|    4.8|          1.5|
|         3|   -0.9|    3.8|          0.3|
|         4|    2.9|    8.2|          9.3|
|         5|    7.3|    8.0|         31.5|
|         6|    4.0|    7.5|          5.3|
|         7|    2.4|    5.5|          6.1|
|         8|    3.4|    6.0|         11.7|
|         9|    0.5|    3.7|          0.0|
|        10|    1.1|    5.2|          0.4|
|        11|    3.4|    7.6|          7.4|
|        12|    1.9|    3.9|          0.1|
|        13|    0.7|    6.7|         17.8|
|        14|   -2.2|    2.2|          6.6|
|        15|   -0.7|    6.0|          5.0|
|        16|    1.6|    6.4|         12.9|
|        17|   -1.0|    2.2|          8.2|
|        18|   -3.6|    4.2|          3.9|
|        19|    0.9|    4.3|          4.5|
|        20|   -1.2|    3.3|          2.1|
+----------

## Part 4: Executing a SQL query
So far in the lab, you’ve successfully managed to load CSV data into a DataFrame object. The whole reason for doing this is so that you can execute Spark SQL queries on the data. This is what you’ll do now…

Follow these steps:
* On your DataFrame object, call the createOrReplaceTempView() method to create a temporary view in Hive MetaStore. This enables you to execute SQL statements on the data.
* On your GlueContext object, call the sql() method to invoke some SQL. For example, select the DayOfMonth, MaxTemp, and MinTemp columns for each record. Display the results returned by the sql() method.

Run your code and verify it works.

In [33]:
# Add code for part 4
df.createOrReplaceTempView("weather")

result = glueContext.sql("FROM weather SELECT DayOfMonth, MaxTemp, MinTemp")
result.show()


+----------+-------+-------+
|DayOfMonth|MaxTemp|MinTemp|
+----------+-------+-------+
|         1|    9.7|    3.1|
|         2|    4.8|    0.6|
|         3|    3.8|   -0.9|
|         4|    8.2|    2.9|
|         5|    8.0|    7.3|
|         6|    7.5|    4.0|
|         7|    5.5|    2.4|
|         8|    6.0|    3.4|
|         9|    3.7|    0.5|
|        10|    5.2|    1.1|
|        11|    7.6|    3.4|
|        12|    3.9|    1.9|
|        13|    6.7|    0.7|
|        14|    2.2|   -2.2|
|        15|    6.0|   -0.7|
|        16|    6.4|    1.6|
|        17|    2.2|   -1.0|
|        18|    4.2|   -3.6|
|        19|    4.3|    0.9|
|        20|    3.3|   -1.2|
+----------+-------+-------+
only showing top 20 rows


## Part 5 (If Time Permits): Generating computed columns
Spark SQL allows you to add computed columns to a DataFrame. For example, you could add a column named DiurnalRange that contains the difference between the maximum and minimum temperature for each day.

One way to add a column to a DataFrame is to call the withColumn() method. Pass in two parameters as follows:
* The name of the column you want to add, e.g., "DiurnalRange".
* The value you want to compute, i.e., the difference between the MaxTemp column and the MinTemp column. You can access a column value via the col() function, e.g. col("MaxTemp") will give you the value of the MaxTemp column. The col() function is defined in the pyspark.sql.functions module, so you’ll need to import it.

Display the DataFrame to verify that it now has a DiurnalRange column. Consider how you can improve the output, e.g. via the round() function in the pyspark.sql.functions module.

In [36]:
# Add code for part 5 
from pyspark.sql.functions import col, round

df.createOrReplaceTempView("weather")

result = glueContext.sql("FROM weather SELECT DayOfMonth, MaxTemp, MinTemp") \
                   .withColumn("DiurnalRange", round(col("MaxTemp") - col("MinTemp"), 2))
result.show()

result2 = glueContext.sql("FROM weather SELECT DayOfMonth, MaxTemp, MinTemp, ROUND(MaxTemp - MinTemp, 2) as DiurnalRange")
result2.show()



+----------+-------+-------+------------+
|DayOfMonth|MaxTemp|MinTemp|DiurnalRange|
+----------+-------+-------+------------+
|         1|    9.7|    3.1|         6.6|
|         2|    4.8|    0.6|         4.2|
|         3|    3.8|   -0.9|         4.7|
|         4|    8.2|    2.9|         5.3|
|         5|    8.0|    7.3|         0.7|
|         6|    7.5|    4.0|         3.5|
|         7|    5.5|    2.4|         3.1|
|         8|    6.0|    3.4|         2.6|
|         9|    3.7|    0.5|         3.2|
|        10|    5.2|    1.1|         4.1|
|        11|    7.6|    3.4|         4.2|
|        12|    3.9|    1.9|         2.0|
|        13|    6.7|    0.7|         6.0|
|        14|    2.2|   -2.2|         4.4|
|        15|    6.0|   -0.7|         6.7|
|        16|    6.4|    1.6|         4.8|
|        17|    2.2|   -1.0|         3.2|
|        18|    4.2|   -3.6|         7.8|
|        19|    4.3|    0.9|         3.4|
|        20|    3.3|   -1.2|         4.5|
+----------+-------+-------+------